In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import tweepy
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [34]:
dataframe = pd.read_csv('TweetsCOV19_3.tsv', sep='\t', error_bad_lines=False )

b'Skipping line 9885107: expected 12 fields, saw 13\n'


In [35]:
#Rename columns
dataframe.columns = ['id','username','date','followers','friends','retweets','favorites','entities','sentiment','mentions','hashtags','url']

In [36]:
#Remove all rows with nan and 'null;' values in column hashtags and entities
df = dataframe[dataframe['hashtags'].notna()]
df = df.loc[df['hashtags']!='null;',]
df = df.loc[df['entities']!='null;',]
df = df.reset_index()

In [37]:
#New dataframe that will be filled
df_clean = pd.DataFrame(columns=df.columns)

In [38]:
#Index of the rows which satisfy the hashtag requirements
index = np.array([])
for i in range(len(df)):
    if ('covid19' in df['hashtags'][i]) or ('coronavirus' in df['hashtags'][i]) or ('covid' in df['hashtags'][i]) or ('covaccine' in df['hashtags'][i]) or ('lockdown' in df['hashtags'][i]) or ('homequarantine' in df['hashtags'][i]) or ('quarantinecenter' in df['hashtags'][i]) or ('socialdistancing' in df['hashtags'][i]) or ('stayhome' in df['hashtags'][i]) or ('staysafe' in df['hashtags'][i]):
        index = np.append(index, i)

In [39]:
#Fill the New dataframe with rows == index
df_clean = df[df.index.isin(index)]
df_clean = df_clean.reset_index()
#Drop the column 'username'
df_clean = df_clean.drop('username', axis=1)

In [40]:
#Save the dataframe as a CSV to improve the efficiency 
df_clean.to_csv('df3.csv')

In [2]:
#CHECKPOINT - Load the dataframe
df = pd.read_csv('df3.csv')
df = df.drop('Unnamed: 0', axis=1)

In [52]:
#TWITTER API in order to download the text of every tweet
consumer_key = "uSxZPLHGWcGBE9TCeawvoK0TN"
consumer_secret = "YBLPpGGJ2zPDKyvFaY1Kz9aEYBToINZpaOAjOhB8ZrgxEigUhY"
access_token = "1481296108492046337-EWsoOM8A8YSO1X0uFk0EbuIIMMEcW9"
access_token_secret = "x5UZQLD24klamFOWliTTJXp9P3L8lTbHomBWk09q9VSCg"

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)
  
# calling the api 
api = tweepy.API(auth)

lista_tweets = {}
# list of status IDs to be fetched 
for i in range(0,len(df),100):
    id_ = list(df['id'].values)[i:i+100]
    # fetching the statuses
    statuses = api.lookup_statuses(id_, tweet_mode='extended')
    
    for status in statuses:
        lista_tweets[status.id_str] = status.full_text

In [57]:
#Remove all rows whitout text
c = 0
for i in tqdm(df['id']):
    if (str(i) in lista_tweets.keys()) == False:
        df = df.drop(c,axis=0)
    c += 1

100%|██████████| 147941/147941 [40:10<00:00, 61.36it/s] 


In [58]:
df = df.drop('level_0', axis=1)
df = df.reset_index()

In [59]:
#Order the dictionary likes the dataframe
sorted_dict = OrderedDict([(el, lista_tweets[str(el)]) for el in list(df['id'])])

In [60]:
#Create new column with all texts
df['Text'] = list(sorted_dict.values())

In [61]:
#Save the dataframe in order to improve the efficiency
df.to_csv('df3_text.csv')

In [24]:
#CHECKPOINT
df = pd.read_csv('df3_text.csv')
df = df.drop('Unnamed: 0', axis=1)

In [25]:
#Cleaner
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

clean_text = []
for or_text in tqdm(df['Text']):
    text = or_text
    
    #Remove links from the corpus of every tweet
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    
    #Remove emoticons from the corpus of every tweet
    text = remove_emoji(text)
    
    #Remove hashtags from the corpus of every tweet
    new_text = []
    for word in text.split():
        if (('#' in word) == False) and (('@' in word) == False):
            new_text.append(word)
    text = ' '.join(new_text)
    
    #Remove stop words from the corpus of every tweet
    sw_nltk = stopwords.words('english')
    words = [word for word in text.split() if word.lower() not in sw_nltk]
    text = ' '.join(words)

    #Remove punctuation from the corpus of every tweet
    punc = '''!()-[]{};:'"\,<>./?!@’#+$%^&*_~“”'''
    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
    
    #Remove some words from the corpus of every tweet
    no_words = ['null','https','don','i','it','My','to','and','amp','s','t','don','m','us','can','not','l','2','we','re','you','I','ve','“','”','r','it','|','•','I']
    for word in text.split():
        if word in no_words:
            text = text.replace(word, '')
    
    text = ' '.join(text.split())
    
    clean_text.append(text)

100%|██████████| 47726/47726 [00:18<00:00, 2571.85it/s]


In [26]:
df['clean'] = clean_text

In [27]:
#Save the final df cleaned
df.to_csv('df3_cleaned.csv')